# Get Prices

In this notebook, we compile a list of historic reserve prices for the different tokens in Aave.

In [1]:
#import packages
import requests
import json
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np
import matplotlib.dates as md
import math
import time

In C:\Users\CCCam\Miniconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\CCCam\Miniconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\CCCam\Miniconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\CCCam\Miniconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\CCCam\Miniconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_tes

We begin by loading in a list of the reserves available in Aave.

In [2]:
#get list of unique reserves
df_reserves = pd.read_csv('tokens.csv')
reserves = list(df_reserves['symbol'].unique())
reserves

['TUSD',
 'AmmUniWBTCUSDC',
 'RAI',
 'GUSD',
 'YFI',
 'BAT',
 'MANA',
 'DPI',
 'AmmBptWBTCWETH',
 'UNI',
 'AmmWBTC',
 'WBTC',
 'AmmUniYFIWETH',
 'AmmUniCRVWETH',
 'REN',
 'AmmUniSNXWETH',
 'BUSD',
 'AmmGUniDAIUSDC',
 'LINK',
 'SUSD',
 'AmmBptBALWETH',
 'AmmDAI',
 'DAI',
 'AAVE',
 'FRAX',
 'XSUSHI',
 'AmmUniRENWETH',
 'PAX',
 'FEI',
 'MKR',
 'AmmUSDC',
 'USDC',
 'AmmUniLINKWETH',
 'AmmUniDAIWETH',
 'AmmUniDAIUSDC',
 'AmmUniUSDCWETH',
 'AmmUniBATWETH',
 'BAL',
 'AmmUniWBTCWETH',
 'SNX',
 'AmmWETH',
 'WETH',
 'AmmUniMKRWETH',
 'AmmGUniUSDCUSDT',
 'AmmUniUNIWETH',
 'AMPL',
 'RENFIL',
 'CRV',
 'AmmUSDT',
 'USDT',
 'KNC',
 'AmmUniAAVEWETH',
 'ZRX',
 'ENJ']

If this data was previously compiled, we load the existing data.

In [3]:
start_time = 0
try:
    old_data = pd.read_csv('prices.csv')
    exists = True
    start_time = old_data['timestamp'].max()
    old_data.info()
except:
    old_data = pd.DataFrame()
    exists = False

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 497589 entries, 0 to 497588
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   price      497589 non-null  object 
 1   timestamp  497589 non-null  int64  
 2   symbol     497589 non-null  object 
 3   priceETH   497589 non-null  float64
 4   datetime   497589 non-null  object 
 5   priceUSD   497589 non-null  float64
dtypes: float64(2), int64(1), object(3)
memory usage: 22.8+ MB


C:\Users\CCCam\Miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3326: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Next, we write a query to gather price data for each reserve from the API.

In [4]:
i=1
all_data= pd.DataFrame()

#get price data for each reserve
for reserve in reserves:
    asset = '"'+reserve+'"'
    lastTime=start_time
    price_data=[]
    #loop until no data left
    while(1):
        try:
            #set query
            query="""
                {
              reserves(where:{symbol: """+asset+"""}){
                symbol
                price{
                  priceHistory(first: 1000 orderBy: timestamp where:{timestamp_gt:"""+str(lastTime)+""" }){
                    price
                    timestamp
                  }
                }
              }
            }    
                """
            #make request
            url = 'https://api.thegraph.com/subgraphs/name/aave/protocol-v2'
            request = requests.post(url,json={'query':query})
            #store data
            price_data.extend(request.json()['data']['reserves'][0]['price']['priceHistory'])
            lastTime = str(request.json()['data']['reserves'][0]['price']['priceHistory'][-1]['timestamp'])
        except Exception as e:
            #exit when no more data left to get
            break
    #skip reserves with no price data
    if len(price_data)==0:
        continue
    df_token_prices = pd.DataFrame(price_data)
    df_token_prices['symbol']=reserve
    df_token_prices['priceETH']=df_token_prices['price'].astype(float)/(1e18)
    all_data = all_data.append(df_token_prices)
    print(str(i)+'/'+str(len(reserves)-2))
    i+=1
all_data.info()

1/52
2/52
3/52
4/52
5/52
6/52
7/52
8/52
9/52
10/52
11/52
12/52
13/52
14/52
15/52
16/52
17/52
18/52
19/52
20/52
21/52
22/52
23/52
24/52
25/52
26/52
27/52
28/52
29/52
30/52
31/52
32/52
<class 'pandas.core.frame.DataFrame'>
Int64Index: 6439 entries, 0 to 110
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   price      6439 non-null   object 
 1   timestamp  6439 non-null   int64  
 2   symbol     6439 non-null   object 
 3   priceETH   6439 non-null   float64
dtypes: float64(1), int64(1), object(2)
memory usage: 251.5+ KB


We calculate the price in USD for each row.

In [5]:
#append new data to old data
all_data = old_data.append(all_data)
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 504028 entries, 0 to 110
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   price      504028 non-null  object 
 1   timestamp  504028 non-null  int64  
 2   symbol     504028 non-null  object 
 3   priceETH   504028 non-null  float64
 4   datetime   497589 non-null  object 
 5   priceUSD   497589 non-null  float64
dtypes: float64(2), int64(1), object(3)
memory usage: 26.9+ MB


In [6]:
all_data['datetime'] = pd.to_datetime(all_data['timestamp'],unit='s')

In [7]:
i=0
#get price in USD for each price udate
def toUSD(row):
    #if old data
    try:
        return row['priceUSD']
    #if new data
    except:
        global i
        USDT = all_data[(all_data['symbol']=='USDT') &(all_data['timestamp']<=row['timestamp'])].sort_values('timestamp')
        ETH = list(USDT['priceETH'])[-1]
        i+=1
        if i%1000==0:
            print(i)
        return row['priceETH']/ETH

all_data['priceUSD'] = all_data.apply(lambda x: toUSD(x),axis=1)
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 504028 entries, 0 to 110
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   price      504028 non-null  object        
 1   timestamp  504028 non-null  int64         
 2   symbol     504028 non-null  object        
 3   priceETH   504028 non-null  float64       
 4   datetime   504028 non-null  datetime64[ns]
 5   priceUSD   497589 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(2)
memory usage: 26.9+ MB


Next, we append WETH data.

In [8]:
WETH = all_data[all_data['symbol']=='USDT'].copy()
WETH['priceUSD'] = 1/WETH['priceETH']
WETH['priceETH']=1
WETH['symbol']='WETH'
WETH.tail()

,price,timestamp,symbol,priceETH,datetime,priceUSD
407,345676893562401,1646283073,WETH,1,2022-03-03 04:51:13,2892.874874
408,342216731893745,1646307086,WETH,1,2022-03-03 11:31:26,2922.124802
409,346221207942428,1646319284,WETH,1,2022-03-03 14:54:44,2888.326818
410,349819900000000,1646320852,WETH,1,2022-03-03 15:20:52,2858.613818
411,353505942021968,1646322442,WETH,1,2022-03-03 15:47:22,2828.806764


In [9]:
all_data = all_data.append(WETH)
all_data.tail()

,price,timestamp,symbol,priceETH,datetime,priceUSD
407,345676893562401,1646283073,WETH,1.0,2022-03-03 04:51:13,2892.874874
408,342216731893745,1646307086,WETH,1.0,2022-03-03 11:31:26,2922.124802
409,346221207942428,1646319284,WETH,1.0,2022-03-03 14:54:44,2888.326818
410,349819900000000,1646320852,WETH,1.0,2022-03-03 15:20:52,2858.613818
411,353505942021968,1646322442,WETH,1.0,2022-03-03 15:47:22,2828.806764


Finally, we save the data to a csv file.

In [10]:
all_data.to_csv("prices.csv",index=False)
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 534650 entries, 0 to 411
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   price      534650 non-null  object        
 1   timestamp  534650 non-null  int64         
 2   symbol     534650 non-null  object        
 3   priceETH   534650 non-null  float64       
 4   datetime   534650 non-null  datetime64[ns]
 5   priceUSD   528211 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(2)
memory usage: 28.6+ MB
